In [296]:
# Import google gemini llm, PromptTemplate, and structured output parser
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os, uuid
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Here we import necessary lib for structure output
from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel, Field

import json
from langchain_community.utilities import GoogleSerperAPIWrapper
import requests
from langchain.agents import create_react_agent, AgentExecutor
from langchain.output_parsers import PydanticOutputParser


from langchain.tools import StructuredTool
from langchain_core.tools import tool

# from newsapi import NewsApiClient
from datetime import datetime, timedelta


import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [297]:
load_dotenv()

True

In [298]:
model=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite",temperature=0.2)


In [299]:
SERPER_API_KEY=os.getenv("SERPER_API_KEY")
NEWS_API_KEY = os.getenv("NEWS_API_KEY")
MAILJET_API_KEY=os.getenv("MAILJET_API_KEY")
MAILJET_API_SECRET=os.getenv("MAILJET_API_SECRET")

In [300]:
ARTIFACT_STORE: dict[str, dict[str, any]] = {}
def save_artifact(prefix: str, obj: any) -> str:
    aid = f"{prefix}_{uuid.uuid4().hex[:8]}"
    ARTIFACT_STORE[aid] = obj
    return aid

In [301]:
LAST_IDS = {"news": None, "sum": None, "sent": None}
RUN_STATE = {"email_sent": False}

def reset_state():
    LAST_IDS.update({"news": None, "sum": None, "sent": None})
    RUN_STATE["email_sent"] = False

In [302]:

@tool(response_format="content_and_artifact")
def fetch(company:str)->tuple[str, dict[str, str]]:
    """This function is used to fetch latest news articles from the Internet about a specific company. The input of this function is the topic on which you want to fetch the news articles. The output SHOULD BE the PYTHON dictionary of latest news articles"""
    
    # It is predefined url from newsapi.org that give latest news articles with everything endpoints. For more information visit https://newsapi.org/docs/endpoints/everything
    present_date=datetime.now().date()
    previous_date=present_date-timedelta(days=2)
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": company,
        "from": previous_date,
        "to": present_date,
        "sortBy": "popularity",
        "language": "en",
        "apiKey": NEWS_API_KEY
    }
    response = requests.get(url, params=params)
    n=1
    articles={}
    response_history={}
    for i in response.json()["articles"][:5]:
        articles[f"Article {n}"]= str(i["title"])+" "+" "+str(i["description"])+" "+i["url"]
        n=n+1
        
    artifact_id = save_artifact("news",articles)
    LAST_IDS["news"]=artifact_id
    
    

    # content: what the LLM can read (short, informational)
    # content = f"Fetched {len(articles)} articles for '{company}', artifact_id='{artifact_id}"
    content= json.dumps({"artifact_id":artifact_id, "count": len(articles)})
    return content, articles
        
    

In [303]:
class summary_font(BaseModel):
    headline: str = Field(description="Headline of the news article.")
    summarize_text: str =Field(description="2-3 sentence summary focused on impact to companies/crypto/markets.")
summary_parser=PydanticOutputParser(pydantic_object=summary_font)
    
@tool(response_format="content_and_artifact")
def summarize_text(artifact_id:str)->tuple[str,dict[str,dict[str,str]]]:
    """This function SHOULD BE summarise the news articles FOR EACH INDIVIDUAL NEWS ARTICLES. The input of this function can be dictionary or string of news articles. The output SHOULD BE A PYTHON DICTIONARY that have  summary of EACH INDIVIDUAL NEWS ARTICLES's descriptions and key points of the news articles. SHOULD NOT RETURN A PYTHON STRING as output."""
    
    if isinstance(artifact_id,dict):
        artifact_id=ARTIFACT_STORE["artifact_id"]
    # artifact_id=artifact_id["artifact_id"]   
    if artifact_id not in ARTIFACT_STORE:
        return f"Error: unknown artifact_id '{artifact_id}'.", []
    
    articles_text=ARTIFACT_STORE[artifact_id]

            
            
    prompt=ChatPromptTemplate.from_messages(['system',"You are a helpful assistant that summarizes news articles. And also give key points of the news articles.",
                                            'human',"""You are a concise financial news summarizer. Given an article, and MUST follow this format: {format_instructions},
                                             Article:
                                            {article_text}"""])
    prompt=prompt.partial(format_instructions=summary_parser.get_format_instructions())
    summary: dict[str,dict[str,str]]={}
    
    model_summary=model.with_structured_output(summary_font)
    
    for i,text in articles_text.items():
        formatted_prompt=prompt.format_prompt(article_text=text)
        result=model_summary.invoke(formatted_prompt)
        # summary[i]=[response.content]
        # result=summary_parser.parse(response.content)
        summary[i]={
            "headline":result.headline,
            "summarize_text":result.summarize_text
        }
    sum_id=save_artifact("sum",summary) 
    LAST_IDS["sum"]=sum_id   
    # return summary
    # content: what the LLM can read (short, informational)
    #content = f"Summarized {len(summary)} articles, artifact_id='{sum_id}'."
    # # artifact: the real Python dict the next tool will consume
    content=json.dumps({"sum_id":sum_id,"count":len(summary)})
    
    return content, summary

In [304]:
# Things that i learned from here, we can not create class inside a @tool, we can't use the partial_variable format with the chatPromptTemplate.

class sentiment_schema(BaseModel):
        sentiment:Literal["Positive","Negative","Neutral"]=Field(description="This is the sentiment of the news articles on market")
        score:float=Field(description="This is the impact score of the News Article")
sentiment_parser=PydanticOutputParser(pydantic_object=sentiment)

@tool(response_format="content_and_artifact")
def sentiment_analysis(artifact_id:str)->tuple[str,any]:
    """This function is used to perform sentiment analysis on EACH INDIVIDUAL news articles. The input of the functiom can be string or dictionary of news articles's summary and the output SHOULD BE  sentiment of EACH INDIVIDAUL news articles from postive, negative or neutral and IMPACT SCORE OF EACH INDIVIDUAL NEWS ARTICLES."""
    prompt=ChatPromptTemplate.from_messages([('system',"You are a helpful financial sentiment analysis assistant who analysis the summary of news articles and provide a sentiment after analysie all the info and also consider your previous knowledge base to give a accurate sentiment."),
                                            ('human',"""    Find sentiment and impact score of EACH INDIVIDUAL NEWS ARTICLES. 
                                             where:- 100 = News will definitly affect the comapny's market and Stock and financial outlook.- 0 = News will not affect anything. 
                                             summary: {summary}, output format: {format_instruction}""")])

    
    if artifact_id not in ARTIFACT_STORE:
        return f"Error: unknown artifact_id '{artifact_id}'.", {}
    summaries= ARTIFACT_STORE[artifact_id]


    prompt=prompt.partial(format_instruction=sentiment_parser.get_format_instructions())
    sentiment={}
    
    model_sentiment=model.with_structured_output(sentiment_schema)
    for i, text in summaries.items():
        formatted_prompt=prompt.format_prompt(summary=text["summarize_text"])
        parsed=model_sentiment.invoke(formatted_prompt)
        # Parse response into dict instead of keeping raw JSON string
        
        sentiment[i] = {
        "sentiment": parsed.sentiment,
        "score": float(parsed.score)
    }
    
    
    sent_id = save_artifact("sent", sentiment)  
    LAST_IDS["sent"]=sent_id 
    # return sentiment        
    # # content: what the LLM can read (short, informational)
    # content = f"Computed sentiment for {len(sentiment)} items, artifact_id='{sent_id}'."
    # # # artifact: the real Python dict the next tool will consume
    content=json.dumps({"sent_id":sent_id,"count":len(sentiment)})
    return content, sentiment

In [305]:
# def extract_id(maybe_id: any) -> str:
#     """Accept raw id, JSON string {'artifact_id': '...'} or dict and return the bare artifact id."""
#     if isinstance(maybe_id, str):
#         s = maybe_id.strip()
#         if s.startswith("{") and s.endswith("}"):
#             try:
#                 return json.loads(s).get("artifact_id", s)
#             except Exception:
#                 return s
#         return s
#     if isinstance(maybe_id, dict):
#         return str(maybe_id.get("artifact_id", ""))
#     return str(maybe_id)

In [306]:
class SendEmailArgs(BaseModel):
    recipient_email: str = Field(description="Destination email address for the report")

@tool(args_schema=SendEmailArgs)
def send_market_sentiment_email(recipient_email:str) -> str:
    """
    Tool: Generates a market sentiment report (final dict) THAT SHOULD CONTAIN ONLY THOSE NEWS ARTICLES WHOSE IMPACT SCORE IS EQUAL TO OR ABOVE 50.0, 
    asks the LLM to format it into an email, 
    and sends the email via Gmail. THE INPUT OF THIS TOOL SHOULD BE A  EMAIL ADDRESS(STRING) AND SHOULD NOT BE A DICTIONARY
    """
    if RUN_STATE.get("email_sent"):
        return f"Email already sent in this run; skipping."

    sent_id = LAST_IDS.get("sent")
    sum_id = LAST_IDS.get("sum")
    if not sent_id or not sum_id:
        return "Error: missing sentiment or summaries; run previous steps first."

    sentiments = ARTIFACT_STORE.get(sent_id, {})
    summaries = ARTIFACT_STORE.get(sum_id, {})
    
    final_dict = {}
    for k, text in sentiments.items():
        if k in summaries and float(text.get("score", 0.0)) >= 50.0:
            final_dict[k] = {
                "summary": summaries[k].get("summarize_text", ""),
                "sentiment": text.get("sentiment", "neutral"),
                "score": float(text.get("score", 0.0)),
            }
    if not final_dict:
        return "No articles above threshold; email not prepared."

    # Step 2: Ask LLM to write the email body
    prompt = prompt = f"""
You are an expert financial analyst. Write a professional and well-formatted market sentiment email. 
The email should be addressed to an investor, and it must summarize the key news articles and their potential 
impact on the company’s market performance.

** USE YOUR INPUT PARAMETER AND FILL ALL THE REQUIRED FIELDS IN THE EMAIL LIKE THE COMPANY NAME, CURRENT DATE AND TIME. 
   DO NOT LEAVE OR MAKE PLACE HOLDER SPACES IN THE EMAIL. WRITE A PROFESSIONAL LOOKING EMAIL BY USING YOUR INPUT PARAMETERS AND OUTPUT KNOWLEDGE.**
The final report is: {final_dict}
Formatting guidelines(YOU MUST FOLLOW THIS):
1. Start with a short professional greeting and introduction (e.g., "Here is today’s market sentiment report about the company").
2. For each article in the report:
   - Present the information in a clear, professional tone.
   - Use bullet points or numbered sections for readability.
   - Format the details as(CONSIDER ONLY THOSE NEWS ARTICLES WHOSE IMPACT SCORE SHOULD BE ABOVE OR EQUAL TO 50.0):
        • Description / Summary of the News Article  
        • Impact Score (%)  
        • Sentiment (Positive / Neutral / Negative)  
3. End with a short professional closing remark (e.g., "This analysis is intended to help you understand 
   the potential market impact. Please review before making decisions.").
   
"sender_password" is "urff fdmt uufi nvnh"

Note: sender: Mohit Mundria,      department: Market_Sentiment_Project

Ensure the email looks polished, investor-friendly, and ready to be sent as a business email.
"""

    email_body = model.invoke(prompt).content


    # Step 3: Construct email
    msg = MIMEMultipart()
    msg["From"] = "mundriamohit101@gmail.com"
    msg["To"] = recipient_email
    msg["Subject"] = "📊 Market Sentiment Report"
    msg.attach(MIMEText(email_body, "plain"))

    # Step 4: Send via Gmail
    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login("mundriamohit101@gmail.com", "urff fdmt uufi nvnh")
            server.sendmail("mundriamohit101@gmail.com", recipient_email, msg.as_string())
        RUN_STATE["email_sent"] = True
        return {"status": "success"}

    except Exception as e:
        return {"status": "failed", "reason": str(e)}


In [307]:
master_prompt_template=PromptTemplate(template="""
You are a professional financial market sentiment analysis assistant that search for news articles and then prepare a financial report to send via email.  
Your job is to:
1. Search for recent news articles about the company: {company}.  
2. Summarize the news articles with headlines and 2–3 line summaries with the help of summarize_text tool with artifact_id.  
3. Perform sentiment analysis and assign an impact score with the help of sentiment_analysis tool with sent_id.  
4. Prepare a final report that SHOULD contain ONLY THOSE NEWS ARTICLES WHOSE IMPACT SCORE IS EQUAL TO OR ABOVE 50.0  
5. Send the report to the user via email.

Tools: {tools}
When you need to use a tool, use this format:

Thought: Do I need to use a tool?  
Action: the action to take, must be one of [{tool_names}]  
Action Input: the input to the action  
Observation: the result of the action  

REPEAT THIS UNTIL YOU SEND A EMAIL TO RECIPIENT WITH A FINAL REPORT.  


Rules:  
- Always format the email in a **clear, professional, investor-friendly** way.  
- Do not skip any step.  
- Only send filtered results from `final_report`.
- The sender email is: mundriamohit101@gmail.com
- The recipient email is: {recipient_email}
- And app password for the sender email is: urff fdmt uufi nvnh

Now, based on the users request, autonomously execute the workflow by invoking the tools in the correct sequence.

NOTE: ***YOU SHOULD ENSURE THAT YOU SEND ONLY ONE SUCCESSFULLY EMAIL. DO NOT SEND MORE THEN 1 SUCCESSFULL EMAIL TO A SINGLE RECIPIENT.***

Begin!
, {agent_scratchpad}
""")#input_variables=["tools","tool_names","sender_email","recipient_email","password"])

# master_prompt=master_prompt_template.invoke({"tools":[fetch,sentiment_analysis,summarize_text,send_market_sentiment_email],"sender_email":"mundriamohit101gmail.com","recipient_email":"mundriamohit100@gmail.com","password":"urff fdmt uufi nvnh","tool_names":["fetch","sentiment_analysis","summarize_text","send_market_sentiment_email"]})


In [308]:
tools = [fetch, summarize_text, sentiment_analysis, send_market_sentiment_email]
agent = create_react_agent(model, tools,prompt=master_prompt_template)

In [309]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[fetch, summarize_text, sentiment_analysis, send_market_sentiment_email],
    verbose=True,  # Set to True to see tool messages and artifacts in console (e.g., intermediate summaries)
    handle_parsing_errors=True,
    early_stopping_method="force",
    max_iterations=7
)

In [311]:
RUN_STATE["email_sent"] = False
result = agent_executor.invoke({
    "input": "Find news about HAL, summarize them, analyze sentiment and email the report to mundriamohit100@gmail.com.",
    "company":"HAL",
    "recipient_email":"mundriamohit100@gmail.com"
    
})



> Entering new AgentExecutor chain...
Thought: The user wants a financial market sentiment report for the company HAL. I need to fetch news articles, summarize them, perform sentiment analysis, filter based on impact score, and then send the report via email.

First, I need to fetch the news articles for HAL.
Action: fetch
Action Input: HAL{"artifact_id": "news_77b8aaa3", "count": 5}I have fetched the news articles for HAL. The next step is to summarize these articles. I will use the `summarize_text` tool for this purpose. The `artifact_id` from the `fetch` tool is "news_77b8aaa3".
Action: summarize_text
Action Input: news_77b8aaa3{"sum_id": "sum_04ba8b9d", "count": 5}I have summarized the news articles. Now I need to perform sentiment analysis on each of the summarized articles. I will use the `sentiment_analysis` tool for this. The `sum_id` from the `summarize_text` tool is "sum_04ba8b9d".
Action: sentiment_analysis
Action Input: sum_04ba8b9d{"sent_id": "sent_94a0629a", "count": 5}

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15
Please retry in 13.103317354s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 13
}
].
Retrying langchain_google_genai.chat_models._chat_with_retr

The previous observation indicates an "Invalid Format: Missing 'Action:' after 'Thought:'". This suggests that the LLM might have been interrupted or there was an issue in the tool execution flow. However, the last successful action was sending the email. The prompt states: "YOU SHOULD ENSURE THAT YOU SEND ONLY ONE SUCCESSFULLY EMAIL. DO NOT SEND MORE THEN 1 SUCCESSFULL EMAIL TO A SINGLE RECIPIENT." Since the email has already been sent successfully, I should not attempt to send another email.

Therefore, the process is complete.
 THEN 1 SUCCESSFULL EMAIL TO A SINGLE RECIPIENT." Since the email has already been sent successfully, I should not attempt to send another email.

Therefore, the process is complete.
Invalid Format: Missing 'Action Input:' after 'Action:'

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15
Please retry in 4.803215958s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 4
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.

Could not parse LLM output: `The previous observation "Invalid Format: Missing 'Action Input:' after 'Action:'" indicates an issue with the tool execution flow. However, the last successful action was sending the email to `mundriamohit100@gmail.com`. The prompt explicitly states: "YOU SHOULD ENSURE THAT YOU SEND ONLY ONE SUCCESSFULLY EMAIL. DO NOT SEND MORE THEN 1 SUCCESSFULL EMAIL TO A SINGLE RECIPIENT."

Since the email has already been sent successfully, and the goal is to send only one email, the task is complete. I will not attempt to send another email or perform any further actions.
 1 SUCCESSFULL EMAIL TO A SINGLE RECIPIENT."

Since the email has already been sent successfully, and the goal is to send only one email, the task is complete. I will not attempt to send another email or perform any further actions.
`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete response

> Finished chain.
